In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import imageio

In [2]:
image = cv2.imread("flower.png")
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

lower_bound = np.array([50,50,50])
higher_bound = np.array([255,255,255])
mask = cv2.inRange(rgb, lower_bound, higher_bound)

print(mask.shape)
height, width, _ = image.shape

def shiftMask(mask, stride):
    leftMask = np.zeros((height, width))
    rightMask = np.zeros((height, width))
    _, column = mask.shape
    
    for c in range(column):
        if (c >= column-stride):
            rightMask[:, c-column+stride] = mask[:, c]
        else:
            rightMask[:, c+stride] = mask[:, c]
            
    for c in range(column):
        if (c <= stride-1):
            leftMask[:, column-stride+c] = mask[:, c]
        else:
            leftMask[:, c-stride] = mask[:, c]

    return leftMask, rightMask

leftMask, rightMask = shiftMask(mask,3)


(270, 197)


In [3]:
def generateRGB(x):
    R = int(127.5 * math.sin(2*math.pi/1000 * (x+250)) + 127.5)
    G = int(127.5 * math.sin(2*math.pi/1000 * (x)) + 127.5)
    B = int(127.5 * math.sin(2*math.pi/1000 * (x+500)) + 127.5)
    return R,G,B

In [4]:
import imageio
images = []


kernel = np.ones((3,3), np.float32)/9
step = -150
for x in range(5000, 0, step):
    newImage = image.copy()
    newImage[mask == 0] = (127, 127, 127)
    newImage[leftMask > 0] = generateRGB(x+step)
    newImage[rightMask > 0] = generateRGB(x-step)
    newImage[mask > 0] = generateRGB(x)
    dst = cv2.filter2D(newImage,-1,kernel)
    images.append(dst)
imageio.mimsave('flower.gif', images)